<a href="https://colab.research.google.com/github/lakto69/Exploring_Layouts/blob/master/Exif_Imagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de App (1ª parte):

1. [Recebe pasta contendo arquivos de imagem](https://colab.research.google.com/drive/1oenYv80rXnlsSpx1-EIbtIkZorKtL97n#scrollTo=PW5whe68nrGj&line=1&uniqifier=1);
1. [Verifica as coordenadas de cada uma das imagens contidas no `exif`](https://colab.research.google.com/drive/1oenYv80rXnlsSpx1-EIbtIkZorKtL97n#scrollTo=pALsoJo6t4c_);
1. [Cria Pastas *(por país)* e Subpastas *(por cidade)* baseadas em seus dados de coordenadas](https://colab.research.google.com/drive/1oenYv80rXnlsSpx1-EIbtIkZorKtL97n#scrollTo=q9EGnQK4N1wH&line=1&uniqifier=1);
1. [Move os arquivos com informações de coordenadas para as pastas correspondentes e mantém na pasta original as imagens sem informações de coordenadas](https://colab.research.google.com/drive/1oenYv80rXnlsSpx1-EIbtIkZorKtL97n#scrollTo=t89hDRmAVSo-&line=1&uniqifier=1).

# Projeto de App (2ª parte):

1. Renderiza um mapa com cada uma das fotos plotadas, como sendo um ponto, utilizando como referência de local a coordenada contida no `exif` ou o local definido pela pasta/subpasta, caso não haja informação no `exif`;
1. Salva esse mapa em arquivo `html`.

# Projeto de App (3ª parte):

1. Criar uma interface de navegação por linha de tempo + localização;

---

## Pré-requisitos:

In [ ]:
%pip install pycountry
%pip install --upgrade reverse_geocoder
%pip install exifread
%pip install exif
%pip install GPSPhoto
%pip install piexif
%pip install folium

## Recebe pasta contendo arquivos de imagem;

---



In [1]:
file_path = 'com.jpg'

In [ ]:
file_path = 'sem.jpg'

## Usa a biblioteca `GPSPhoto` para acessar dados geográficos de `exif`:

2. https://stackoverflow.com/questions/19804768/interpreting-gps-info-of-exif-data-from-photo-in-python

*Melhor alternativa!*

In [2]:
from GPSPhoto import gpsphoto

In [3]:
# Recebe o endereço de um arquivo e retorna um dicionário com informações de GPS do Exif
data = gpsphoto.getGPSData(file_path)
# print(data.get('Latitude'), data.get('Longitude'))
data

In [4]:
print(gpsphoto.getGPSData('com.jpg'))

None


### Examinando local das imagens com `reverse_geocoder` e `pycountry`:

[https://auth0.com/blog/read-edit-exif-metadata-in-photos-with-python/]

In [5]:
import reverse_geocoder as rg
import pycountry

In [6]:
# Função que recebe coordenadas geográficas e retorna Cidade e País ou None
def latlong2cidade_pais(lati=None, longe=None):
  if lati and longe:
    # Dados geográficos sem Nome do país; só com sigla
    # 'copy()' evita de alterar os dados localizados quando fizer alguma alteração na variável
    location_info = rg.search((lati, longe))[0].copy()
    # Pesquisa o nome do país pela sua sigla
    if pycountry.countries.get(alpha_2=location_info['cc']):
      pais = pycountry.countries.get(alpha_2=location_info['cc']).name
    else:
      pais = None
    return location_info['name'], pais
  else:
    return None, None

In [7]:
print(latlong2cidade_pais(data.get('Latitude'), data.get('Longitude')))

AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
locais = latlong2cidade_pais(data.get('Latitude'), data.get('Longitude'))

print(f'./{locais[1]}/{locais[0]}')

./Brazil/Brasilia


## 3. Criar Pastas (por país) e Subpastas (por cidade) baseadas em seus dados de coordenadas:

In [3]:
import os

In [ ]:
# Busca a cidade e país de uma coordenada e cria as pastas, caso não exista

locais = latlong2cidade_pais(data.get('Latitude'), data.get('Longitude'))
if locais:
  dir = f'./{locais[1]}/{locais[0]}'
else:
  dir = './pais/cidade'       
if not os.path.exists(dir):
  os.makedirs(dir)


## 4. Move os arquivos com informações de coordenadas para as pastas correspondentes e mantém na pasta original as imagens sem informações de coordenadas

In [ ]:
# movendo o arquivo

os.replace(os.path.join('.', file_path), os.path.join(dir, file_path))

### Mostrando a foto no `Google Maps`:

In [ ]:
import webbrowser

In [ ]:
url = f"https://www.google.com/maps?q={decimal_latitude},{decimal_longitude}"
webbrowser.open_new_tab(url)

False



---



# Trabalhando com `Exif` de imagens:

1. Lista de TAGS Exif: [https://exiftool.org/TagNames/EXIF.html]
1. TAGS Exif GPS: [https://exiftool.org/TagNames/GPS.html] e [https://exif.readthedocs.io/en/latest/api_reference.html#image-attributes]
1. Trabalhando com dados de GPS da imagem: [https://auth0.com/blog/read-edit-exif-metadata-in-photos-with-python/]

Fonte: [https://www.geeksforgeeks.org/how-to-extract-image-metadata-in-python/]

In [6]:
# Utilizando `PIL`
# https://acervolima.com/como-extrair-metadados-de-imagem-em-python/
from PIL import Image
from PIL.ExifTags import TAGS

ModuleNotFoundError: No module named 'PIL'

In [ ]:
# open the image
image = Image.open("201901~1.JP8.jpg")

# extracting the exif metadata
exifdata = image.getexif()

# looping through all the tags present in exifdata
for tagid in exifdata:
	
	# getting the tag name instead of tag id
	tagname = TAGS.get(tagid, tagid)

	# passing the tagid to get its respective value
	value = exifdata.get(tagid)

	# printing the final result
	print(f"{tagname:25}: {value}")
	# if tagname == 'GPSInfo':
	print(f'\t{tagname} => {tagid}')


ExifVersion              : b'0220'
	ExifVersion => 36864
ColorSpace               : 1
	ColorSpace => 40961
ApertureValue            : (200, 100)
	ApertureValue => 37378
ExposureMode             : 0
	ExposureMode => 41986
DateTimeDigitized        : 2019:01:01 13:19:06
	DateTimeDigitized => 36868
ExifImageHeight          : 1836
	ExifImageHeight => 40963
SceneCaptureType         : 0
	SceneCaptureType => 41990
SceneType                : b'\x01'
	SceneType => 41729
DigitalZoomRatio         : (100, 100)
	DigitalZoomRatio => 41988
WhiteBalance             : 0
	WhiteBalance => 41987
FocalLength              : (260, 100)
	FocalLength => 37386
ExifImageWidth           : 3264
	ExifImageWidth => 40962
MeteringMode             : 2
	MeteringMode => 37383
UserComment              : b'   HB = 0,6,-6 FN=0 lidx=172.88   FM0   FC000000000\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [ ]:
tag = 34853 # GPSInfo
exifdata.get(tag)

{1: 'N',
 2: ((44, 1), (29, 1), (381710, 10000)),
 3: 'E',
 4: ((11, 1), (20, 1), (335709, 10000)),
 5: (200, 100),
 6: (189000, 1000),
 7: ((12, 1), (19, 1), (20, 1)),
 29: '2019:01:01'}

## Usando `exifread`:

*3ª melhor opção até agora!*

In [ ]:
# Pré-requisito:
!pip install exifread

In [ ]:
file_path = '20171207_112856.jpg'

In [ ]:
# FUNÇÕES AUXILIARES PARA LEITURA DE COORDENADAS GEOGRÁFICAS EM "EXIF"

# barrowed from 
# https://gist.github.com/snakeye/fdc372dbf11370fe29eb 
def _convert_to_degress(value):
    """
    Helper function to convert the GPS coordinates stored in the EXIF to degress in float format
    :param value:
    :type value: exifread.utils.Ratio
    :rtype: float
    """
    d = float(value.values[0].num) / float(value.values[0].den)
    m = float(value.values[1].num) / float(value.values[1].den)
    s = float(value.values[2].num) / float(value.values[2].den)

    return d + (m / 60.0) + (s / 3600.0)


def getGPS(filepath):
    '''
    returns gps data if present other wise returns empty dictionary
    '''
    with open(filepath, 'rb') as f:
        tags = ef.process_file(f)
        latitude = tags.get('GPS GPSLatitude')
        latitude_ref = tags.get('GPS GPSLatitudeRef')
        longitude = tags.get('GPS GPSLongitude')
        longitude_ref = tags.get('GPS GPSLongitudeRef')
        if latitude:
            lat_value = _convert_to_degress(latitude)
            if latitude_ref.values != 'N':
                lat_value = -lat_value
        else:
            return {}
        if longitude:
            lon_value = _convert_to_degress(longitude)
            if longitude_ref.values != 'E':
                lon_value = -lon_value
        else:
            return {}
        return {'latitude': lat_value, 'longitude': lon_value}
    return {}

In [ ]:
# Usando a função getGPS:
gps = getGPS(file_path)
print(f"Europa: {gps['latitude']},{gps['longitude']}")

Europa: -15.806131083333334,-47.88481569444444


In [ ]:
with open(file_path, 'rb') as f:
  tags = ef.process_file(f)

# Lista de TAGs disponíveis no arquivo de exemplo:
print(tags.keys())

dict_keys(['Image DateTime', 'GPS GPSDate', 'GPS GPSAltitudeRef', 'GPS GPSLongitudeRef', 'GPS GPSLongitude', 'GPS GPSLatitudeRef', 'GPS GPSTimeStamp', 'GPS GPSAltitude', 'GPS GPSLatitude', 'Image GPSInfo', 'Image Model', 'Image YCbCrPositioning', 'Image ResolutionUnit', 'Image YResolution', 'Image Orientation', 'Image ExifOffset', 'Image XResolution', 'Image Make', 'Thumbnail YResolution', 'Thumbnail Orientation', 'Thumbnail JPEGInterchangeFormatLength', 'Thumbnail JPEGInterchangeFormat', 'Thumbnail Compression', 'Thumbnail ResolutionUnit', 'Thumbnail XResolution', 'EXIF ColorSpace', 'EXIF DateTimeDigitized', 'EXIF FNumber', 'EXIF FocalLength', 'EXIF ApertureValue', 'EXIF ExposureMode', 'EXIF SubSecTimeDigitized', 'EXIF ExifImageLength', 'EXIF SceneCaptureType', 'EXIF SceneType', 'EXIF SubSecTimeOriginal', 'EXIF DigitalZoomRatio', 'EXIF ExposureProgram', 'EXIF WhiteBalance', 'EXIF ExifImageWidth', 'EXIF SubSecTime', 'EXIF ShutterSpeedValue', 'EXIF MeteringMode', 'EXIF DateTimeOriginal'

In [ ]:
dir(gps)

### Examinando local das imagens com `reverse_geocoder` e `pycountry`:

[https://auth0.com/blog/read-edit-exif-metadata-in-photos-with-python/]

In [ ]:
!pip install pycountry

In [ ]:
!pip install reverse_geocoder

In [ ]:
import reverse_geocoder as rg
import pycountry

In [ ]:
print(f"Location info - Image {file_path}")
print("-----------------------")
decimal_latitude = gps['latitude']
decimal_longitude = gps['longitude']

location_info = rg.search((decimal_latitude, decimal_longitude))[0]
print(location_info)
# location_info['country'] = pycountry.countries.get(alpha_2=location_info['cc'])
# print(f"\n\t{pycountry.countries.get(alpha_2=location_info['cc'])}\n")
print(f"{location_info}\n")

Location info - Image 201901~1.JP8.jpg
-----------------------
{'lat': '44.49381', 'lon': '11.33875', 'name': 'Bologna', 'admin1': 'Emilia-Romagna', 'admin2': 'Provincia di Bologna', 'cc': 'IT', 'country': Country(alpha_2='IT', alpha_3='ITA', flag='🇮🇹', name='Italy', numeric='380', official_name='Italian Republic')}
{'lat': '44.49381', 'lon': '11.33875', 'name': 'Bologna', 'admin1': 'Emilia-Romagna', 'admin2': 'Provincia di Bologna', 'cc': 'IT', 'country': Country(alpha_2='IT', alpha_3='ITA', flag='🇮🇹', name='Italy', numeric='380', official_name='Italian Republic')}



In [ ]:
location_info['country']

Country(alpha_2='IT', alpha_3='ITA', flag='🇮🇹', name='Italy', numeric='380', official_name='Italian Republic')

In [ ]:
pycountry.countries.get(alpha_2=location_info['cc'])

Country(alpha_2='IT', alpha_3='ITA', flag='🇮🇹', name='Italy', numeric='380', official_name='Italian Republic')

### Mostrando a foto no `Google Maps`:

In [ ]:
import webbrowser

In [ ]:
url = f"https://www.google.com/maps?q={decimal_latitude},{decimal_longitude}"
webbrowser.open_new_tab(url)

False



---


## RASCUNHOS...

Utilizando a biblioteca `exif `:

1. https://towardsdatascience.com/read-and-edit-image-metadata-with-python-f635398cd991

1. https://github.com/kennethleungty/Image-Metadata-Exif/blob/main/Image_Metadata_Extraction_EXIF.ipynb

1. https://exif.readthedocs.io/en/latest/api_reference.html

In [ ]:
!pip install exif

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.9 MB/s 


In [ ]:
from exif import Image

# folder_path = 'sample_images'
# img_filename = 'image_1.jpg'
img_path = file_path

with open(img_path, 'rb') as img_file:
    img = Image(img_file)
    
print(img.has_exif)

True


In [ ]:
# List all EXIF tags contained in the image
sorted(img.list_all()), sorted(img.get_all())

In [ ]:
dir(img)

In [ ]:
img.exposure_time, img.datetime, img.datetime_digitized, img.datetime_original

(0.003215434083601286,
 '2019:01:01 13:19:06',
 '2019:01:01 13:19:06',
 '2019:01:01 13:19:06')

In [ ]:
print(img.gps_altitude)
print(img.gps_altitude_ref)
print(img.gps_datestamp)
print(img.gps_latitude)
print(img.gps_latitude_ref)
print(img.gps_longitude)
print(img.gps_longitude_ref)
print(img.gps_timestamp)

189.0
2.0
2019:01:01
(44.0, 29.0, 38.171)
N
(11.0, 20.0, 33.5709)
E
(12.0, 19.0, 20.0)


In [ ]:
print(img.get('gps_speed')) # => GET() retorna None, caso não exista aquela chave

None


### Usando a biblioteca `GPSPhoto`:

1. https://stackoverflow.com/questions/19804768/interpreting-gps-info-of-exif-data-from-photo-in-python

*Melhor alternativa!*

In [ ]:
!pip install GPSPhoto
!pip install piexif

In [ ]:
from GPSPhoto import gpsphoto
# Get the data from image file and return a dictionary
data = gpsphoto.getGPSData(file_path)
print(data['Latitude'], data['Longitude'])

-15.806131083333334 -47.88481569444444


In [ ]:
data

{'Date': '12/07/2017',
 'Longitude': -47.88481569444444,
 'UTC-Time': '13:28:50',
 'Altitude': 0,
 'Latitude': -15.806131083333334}



---



# Trabalhando com Gráficos de Mapa Utilizando o `folium`:

1. https://www.youtube.com/watch?v=EElW-W0GmrA&ab_channel=Stack

In [ ]:
!pip install folium

In [ ]:
import folium

In [ ]:
mapa = folium.Map(location=[data['Latitude'], data['Longitude']])

In [ ]:
mapa

### Especificando diversos estilos:

Conjunto de `tiles` que podem ser usados:

1. ”OpenStreetMap”
1. ”Stamen Terrain”
1. “Stamen Toner”
1. “Stamen Watercolor”
1. ”CartoDB positron”
1. “CartoDB dark_matter”

In [ ]:
folium.Map(
    location=[-19,-43],
    tiles='Stamen Toner'
)

In [ ]:
folium.Map(
    location=[-19,-43],
    tiles='Stamen Terrain'
)

In [ ]:
mapa = folium.Map(
    location=[-19.916667,-43.9333333],
    tiles='Stamen Terrain',
    zoom_start=14
)

### Adicionando marcadores:

In [ ]:
folium.Marker(
    [-19.9559872,-43.9151621],
    popup='<i>Praça do Papa<\i>',
    tooltip='Clique aqui',
    icon=folium.Icon(color='red')
).add_to(mapa)

### Adicionando Círculos de determinado tamanho em metros

In [ ]:
folium.CircleMarker(
    [-19.9559872,-43.9151621],
    radius=50,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(mapa)
mapa

### Rascunho:

In [ ]:
with open('Baobás de Brasília.kml') as f:
  kml = f.read()

In [ ]:
m = folium.Map(location=[45.5236, -122.6750], tiles="Stamen Toner", zoom_start=13)

folium.Circle(
    radius=100,
    location=[45.5244, -122.6699],
    popup="<img src='https://www.homehost.com.br/blog/wp-content/uploads/2022/10/Como-Ativar-e-Configurar-o-filtro-de-Spam-usando-o-cPanel..jpg' title='The Waterfront'>",
    color="crimson",
    fill=False,
).add_to(m)

folium.CircleMarker(
    location=[45.5215, -122.6261],
    radius=50,
    popup='<img src="/content/com.jpg" title="Logo da HomeHost">',  #"Laurelhurst Park",
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
).add_to(m)


m

### Criando pontos com o clique do mouse

In [ ]:
m = folium.Map(location=[46.8527, -121.7649], tiles="Stamen Terrain", zoom_start=13)

folium.Marker([46.8354, -121.7325], popup="Camp Muir").add_to(m)

m.add_child(folium.ClickForMarker(popup="Waypoint"))


m

In [ ]:
import json

import requests

url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
vis1 = json.loads(requests.get(f"{url}/vis1.json").text)
vis2 = json.loads(requests.get(f"{url}/vis2.json").text)
vis3 = json.loads(requests.get(f"{url}/vis3.json").text)

In [ ]:
m = folium.Map(location=[46.3014, -123.7390], zoom_start=7, tiles="Stamen Terrain")

folium.Marker(
    location=[47.3489, -124.708],
    popup=folium.Popup(max_width=450).add_child(
        folium.Vega(vis1, width=450, height=250)
    ),
).add_to(m)

folium.Marker(
    location=[44.639, -124.5339],
    popup=folium.Popup(max_width=450).add_child(
        folium.Vega(vis2, width=450, height=250)
    ),
).add_to(m)

folium.Marker(
    location=[46.216, -124.1280],
    popup=folium.Popup(max_width=450).add_child(
        folium.Vega(vis3, width=450, height=250)
    ),
).add_to(m)


m

In [ ]:
pontos = localizacoes_geo[['latitude', 'longitude']].values.tolist()
for point in range(0, len(pontos)):
    folium.Marker(locationlist[point]).add_to(mapa)
mapa

In [ ]:
geo_json_mapa['features'][0]['properties']['latitude'], geo_json_mapa['features'][0]['properties']['longitude']

for i in range(0, len(geo_json_mapa['features'])):
  # print(geo_json_mapa['features'][i]['properties']['latitude'], geo_json_mapa['features'][i]['properties']['longitude'])
  ponto = geo_json_mapa['features'][i]['properties']['latitude'], geo_json_mapa['features'][i]['properties']['longitude']
  folium.Marker(ponto).add_to(mapa)
mapa

ValueError: ignored

---

### Fim de rascunho

### Exibindo as coordenadas do local do clique:

In [ ]:
mapa.add_child(folium.LatLngPopup())

### Adicionando diversos pontos no mapa:

### Salvando o mapa resultante em um arquivo `html`:

In [ ]:
mapa.save('meu_mapa.html')

https://python-visualization.github.io/folium/modules.html:

In [ ]:
m = folium.Map(location=[45.523, -122.675], width=750, height=500)
m = folium.Map(location=[45.523, -122.675], tiles='cartodb positron')
m = folium.Map(
   location=[45.523, -122.675],
   zoom_start=2,
   tiles='https://api.mapbox.com/v4/mapbox.streets/{z}/{x}/{y}.png?access_token=mytoken',
   attr='Mapbox attribution'
)


In [ ]:
m.fit_bounds([[52.193636, -2.221575], [52.636878, -1.139759]])

In [ ]:
m